In [3]:
#Importing
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar
from sympy import *
import math
from sympy.solvers.solveset import linsolve
from pylab import array
#---------------------------------
#WE USE NATURAL UNITS IN THIS CODE
#---------------------------------

In [2]:
# Global Constants
h =  2*math.pi
kb = 1
me = 511e3 #[eV]

In [3]:
#Ionisation energies (eV)
xi_H1  = 13.6
xi_He1 = 24.6
xi_He2 = 54.4

#Partition Functions (taken from independent solutions for
# each element, so might not be as obvious as they are here...)
zh0  = 2
zh1  = 1
zhe0 = 1
zhe1 = 2
zhe2 = 1

In [4]:
#Total Densities (GIVEN in g/cm^3, transformed to eV&**4)
nH = (0.36209)*4.31013646*(10**18)/(0.93878405573*10**9)
nHe = (0.62194)*4.31013646*(10**18)/(0.372784341*10**(10))

In [5]:
# Temperature (6700K expressed in eV/Kb)
T = (15490000)/11604.51812

In [6]:
#Define the right side of the Saha Equation, which depends on 
# the Temperature [T], the difference in energies of the states [xi]
# and the fraction of the partition functions G(r+1)/G(r) [z0,z1]

def lam(T,xi,z0,z1):
    return (z1/z0) * ((2*math.pi*me*kb*T)**1.5 / (h**3)) * math.exp((-xi) / (kb*T));

In [7]:
#Making the electron density into a symbolic variable
nE = symbols('n_e')

#The matrix representing the system of equations and their value
M = Matrix([[-lam(T,xi_H1,zh0,zh1), nE,                        0,                        0,   0,   0],
            [                    0,  0, -lam(T,xi_He1,zhe0,zhe1),                       nE,   0,   0],
            [                    0,  0,                        0, -lam(T,xi_He2,zhe1,zhe2),  nE,   0],
            [                    1,  1,                        0,                        0,   0,  nH],
            [                    0,  0,                        1,                        1,   1, nHe],
            [                    0,  1,                        0,                        1,   2,  nE]])

expr = M.det()

print(expr)

def eq(ne):
    return expr.subs(nE,ne)

(-3.91772430285992e-47*n_e**12 - 2.84395232316378e-34*n_e**11 - 9.27516192647486e-22*n_e**10 - 1.79637451607351e-9*n_e**9 - 2298.25975129704*n_e**8 - 2.04137083121144e+15*n_e**7 - 1.28545868163522e+27*n_e**6 - 5.74157192631066e+38*n_e**5 - 1.78298929253598e+50*n_e**4 - 3.66446600553884e+61*n_e**3 - 4.47408930680409e+72*n_e**2 - 2.42025089014163e+83*n_e + 7.90971428705038e+92)/(3.91772430285992e-47*n_e**8 + 1.75455511937983e-34*n_e**7 + 3.43778109476446e-22*n_e**6 + 3.84903068263572e-10*n_e**5 + 269.342287719279*n_e**4 + 120625101015495.0*n_e**3 + 3.37637637553303e+25*n_e**2 + 5.40040047731652e+36*n_e + 3.7790195482793e+47)


In [8]:
max_ne = nH + 2*nHe;
print("The maximum electron density:")
print(max_ne)
min_ne = 0
sol = root_scalar(eq, method='brentq', bracket=(min_ne,max_ne))
print(sol)
print("\nThe ionisation percentage:")
print((sol.root * 100)/max_ne)

The maximum electron density:
3100599536.992421
      converged: True
           flag: 'converged'
 function_calls: 6
     iterations: 5
           root: 3087404396.3414354

The ionisation percentage:
99.57443260590225


In [9]:
M.subs(nE,sol.root)

Matrix([
[-559813230763.524, 3087404396.34144,                 0,                 0,                0,                0],
[                0,                0, -2220875555807.54,  3087404396.34144,                0,                0],
[                0,                0,                 0, -542960951771.586, 3087404396.34144,                0],
[                1,                1,                 0,                 0,                0, 1662424176.54594],
[                0,                0,                 1,                 1,                1, 719087680.223242],
[                0,                1,                 0,                 1,                2, 3087404396.34144]])

In [10]:
#Now we know the value of ne, we compute the equations
nh0, nh1, nhe0, nhe1, nhe2 = symbols('nh0, nh1, nhe0, nhe1, nhe2')
valuez = solve_linear_system(M, nh0, nh1, nhe0, nhe1, nhe2)
print(valuez)

None


In [16]:
#try converting to numpy matrix
MM = np.array(np.array(M.subs(nE,sol.root)), np.float64)
print(MM)

#Then split into system of equations and results(not overdetermined so only taking up tp 5 rows)


[[-5.59813231e+11  3.08740440e+09  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.22087556e+12  3.08740440e+09
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.42960952e+11
   3.08740440e+09  0.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.66242418e+09]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  7.19087680e+08]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00
   2.00000000e+00  3.08740440e+09]]


In [22]:
xx = np.linalg.solve(m1, m2)
print(xx)
#So it seems using the overdetermined matrix for solving is ruining the solver,
#this is without that that last equation.

[9.11808477e+06 1.65330609e+09 5.65210398e+03 4.06575167e+06
 7.15016276e+08]


In [4]:
File_data = np.loadtxt("reduced_data.txt", dtype=np.float64)

In [5]:
print(File_data)

[[4.0000e-07 1.5000e-03 1.5490e+07 1.5050e+02 2.3340e+17 0.0000e+00
  3.6209e-01 6.2194e-01 8.6200e-06]
 [9.0000e-07 2.0000e-03 1.5490e+07 1.5050e+02 2.3330e+17 1.0000e-05
  3.6213e-01 6.2189e-01 8.6240e-06]
 [1.7000e-06 2.5000e-03 1.5490e+07 1.5050e+02 2.3330e+17 1.0000e-05
  3.6219e-01 6.2183e-01 8.6290e-06]
 [2.9000e-06 3.0000e-03 1.5490e+07 1.5040e+02 2.3330e+17 2.0000e-05
  3.6226e-01 6.2175e-01 8.6350e-06]
 [4.6000e-06 3.5000e-03 1.5490e+07 1.5040e+02 2.3320e+17 4.0000e-05
  3.6236e-01 6.2165e-01 8.6430e-06]
 [6.8000e-06 4.0000e-03 1.5490e+07 1.5030e+02 2.3320e+17 6.0000e-05
  3.6247e-01 6.2154e-01 8.6510e-06]
 [9.7000e-06 4.5000e-03 1.5480e+07 1.5030e+02 2.3310e+17 8.0000e-05
  3.6259e-01 6.2142e-01 8.6610e-06]
 [1.3300e-05 5.0000e-03 1.5480e+07 1.5020e+02 2.3300e+17 1.1000e-04
  3.6273e-01 6.2128e-01 8.6720e-06]
 [1.7800e-05 5.5000e-03 1.5480e+07 1.5020e+02 2.3290e+17 1.5000e-04
  3.6288e-01 6.2113e-01 8.6840e-06]
 [2.3000e-05 6.0000e-03 1.5480e+07 1.5010e+02 2.3290e+17 1.9000e

In [7]:
File_data[0,0:5]

array([4.000e-07, 1.500e-03, 1.549e+07, 1.505e+02, 2.334e+17])

In [12]:
np.shape(File_data[:,0])

(14,)

In [28]:
rows, columns = File_data.shape
for i in range(rows):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [19]:
print(File_data[0,6])

0.36209


In [11]:
Temps = [];
for i in range(100):
    Temps.append(i*100)
    
Temps[0:3]

[0, 100, 200]